Summation

In [2]:
import numpy as np
import copy
%matplotlib inline

In [3]:
def sigmoid(x):
    y = 1/(1+np.exp(-x))
    return y
def dersigmoid(x):
    y = x*(1-x)
    return y

In [4]:
#Initialize the model
step = 0.1
ipnumber = 2
hdnumber = 32
opnumber = 2

neu_i2h = 2*np.random.random((ipnumber,hdnumber)) - 1
neu_h2o = 2*np.random.random((hdnumber,opnumber)) - 1
neu_h2h = 2*np.random.random((hdnumber,hdnumber)) - 1

neu_i2hN = np.zeros_like(neu_i2h)
neu_h2oN = np.zeros_like(neu_h2o)
neu_h2hN = np.zeros_like(neu_h2h)

In [5]:
#Set to binary
i2b = {}
bdigit = 8
MAXnumber = pow(2,bdigit)
bin = np.unpackbits(np.array([range(MAXnumber)],dtype=np.uint8).T,axis=1)
for i in range(MAXnumber):
    i2b[i] = bin[i]

In [16]:
for j in range(20000):
    a_decimal = np.random.randint(MAXnumber / 2)
    b_decimal = np.random.randint(MAXnumber / 2)
    c_decimal = a_decimal + b_decimal
    a = i2b[a_decimal]
    b = i2b[b_decimal]
    c = i2b[c_decimal]
    binary = np.zeros_like(c)
    aError = 0
    oplayer_der = list()
    hdlayer_val = list()
    hdlayer_val.append(np.zeros(hdnumber))

    for locate in range(bdigit):
        X = np.array([[a[bdigit - locate - 1],b[bdigit - locate - 1]]])
        Y = np.array([[c[bdigit - locate - 1]]]).T
        hdlayer = sigmoid(np.dot(X,neu_i2h) + np.dot(hdlayer_val[-1],neu_h2h))
        oplayer = sigmoid(np.dot(hdlayer,neu_h2o))
        oplayer_error = Y - oplayer
        oplayer_der.append((oplayer_error)*dersigmoid(oplayer))
        aError += np.abs(oplayer_error[0])

        binary[bdigit - locate - 1] = np.round(oplayer[0][0])

        hdlayer_val.append(copy.deepcopy(hdlayer))

    Fhdlayer_dels = np.zeros(hdnumber)
    for locate in range(bdigit):
        X = np.array([[a[locate],b[locate]]])
        hdlayer = hdlayer_val[-locate-1]
        hdlayer_pre = hdlayer_val[-locate-2]

        oplayer_dels = oplayer_der[-locate-1]
        hdlayer_dels = (Fhdlayer_dels.dot(neu_h2h.T) + oplayer_dels.dot(neu_h2o.T)) * dersigmoid(hdlayer)

        neu_h2oN += np.atleast_2d(hdlayer).T.dot(oplayer_dels)
        neu_h2hN += np.atleast_2d(hdlayer_pre).T.dot(hdlayer_dels)
        neu_i2hN += X.T.dot(hdlayer_dels)
        Fhdlayer_dels = hdlayer_dels

    neu_i2h += neu_i2hN * step
    neu_h2o += neu_h2oN * step
    neu_h2h += neu_h2hN * step

    neu_i2hN *= 0
    neu_h2oN *= 0
    neu_h2hN *= 0

In [17]:
print("Error:" + str(aError))
print("Predicted:" + str(binary))
print("True:" + str(c))
value = 0
for index,x in enumerate(reversed(binary)):
    value += x*pow(2,index)
print(str(a_decimal) + " + " + str(b_decimal) + " = " + str(value))

Error:[0.2022322  0.19161575]
Predicted:[1 0 0 0 1 0 1 0]
True:[1 0 0 0 1 0 1 0]
45 + 93 = 138
